In [1]:
# %% -------------------------------------------------
# 📹  CSV → Skeleton Video  (BODY_25, 전 프레임)
# ----------------------------------------------------
import cv2, pandas as pd, numpy as np
from pathlib import Path
from tqdm import tqdm

# 이미 선언된 EDGES · load_csv() 재사용
# -----------------------------------------------
# EDGES  : BODY_25 관절 연결 리스트
# load_csv() : csv → [T,25,3] numpy 배열

EDGES = [
    (0,1),
      (1,2),(2,3),(3,4),          # Right arm
      (1,5),(5,6),(6,7),          # Left  arm
      (1,8),
        (8,9),(9,10),(10,11),     # Right leg
          (11,24),(11,22),(22,23),
        (8,12),(12,13),(13,14),   # Left leg
          (14,21),(14,19),(19,20),
    (0,15),(15,17),
    (0,16),(16,18)
]

def load_csv(csv_path: Path) -> np.ndarray:
    """return arr [T,25,3] (x,y,c)"""
    df = pd.read_csv(csv_path)
    arr = df.values.reshape(len(df), 25, 3)
    return arr


def csv_to_video(csv_path: str|Path,
                 out_mp4 : str|Path = None,
                 fps:int = 30,
                 height:int = 720,
                 width:int  = 720,
                 conf_th:float = .05):
    """
    csv_path : crop_keypoint/<name>.csv
    out_mp4  : 저장 경로 (None → csv와 같은 이름 + '_skel.mp4')
    """
    csv_path = Path(csv_path)
    if out_mp4 is None:
        out_mp4 = csv_path.with_name(csv_path.stem + "_skel.mp4")

    arr = load_csv(csv_path)                     # [T,25,3]
    # ---------- 좌표 정규화(캔버스 매핑) ----------
    xs, ys = arr[:,:,0], arr[:,:,1]
    xmin, xmax = np.nanmin(xs), np.nanmax(xs)
    ymin, ymax = np.nanmin(ys), np.nanmax(ys)
    scale = min((width-40)/(xmax-xmin+1e-5), (height-40)/(ymax-ymin+1e-5))
    off_x, off_y = 20 - xmin*scale, 20 - ymin*scale   # padding 20px

    # ---------- VideoWriter 초기화 --------------
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    vw = cv2.VideoWriter(str(out_mp4), fourcc, fps, (width, height))

    # ---------- 프레임 루프 ---------------------
    for frame in tqdm(arr, desc=out_mp4.name):
        canvas = np.ones((height, width, 3), np.uint8)*255   # white BG
        pts = []
        for x,y,c in frame:
            if c < conf_th:
                pts.append(None); continue
            px, py = int(x*scale + off_x), int(y*scale + off_y)
            pts.append((px,py))
            cv2.circle(canvas, (px,py), 3, (0,0,255), -1)
        # 뼈대 라인
        for i,j in EDGES:
            if i >= len(pts) or j >= len(pts): continue
            if pts[i] and pts[j]:
                cv2.line(canvas, pts[i], pts[j], (0,128,0), 2)
        vw.write(canvas)

    vw.release()
    print(f"🎞️  영상 저장 완료 → {out_mp4}")

# ===== 사용 예 =====
CUR_DIR =  Path(r"D:\golfDataset\dataset\test")               # 노트북 작업 디렉터리

ROOT_DIR = CUR_DIR / "balanced_true"    # t,f 폴더 선택

KEY_DIR = ROOT_DIR / "crop_keypoint"
# KEY_DIR의 랜덤한 CSV 파일을 선택
import random  
csv_files = list(KEY_DIR.glob("*.csv"))
csv_file = random.choice(csv_files)  # 예시로 랜덤 선택
print(f"선택된 CSV 파일: {csv_file.name}")
csv_to_video(csv_file)                # fps·해상도 옵션은 필요 시 수정

선택된 CSV 파일: 20201210_General_116_DOC_A_M30_MS_071_crop.csv


20201210_General_116_DOC_A_M30_MS_071_crop_skel.mp4: 100%|██████████| 193/193 [00:00<00:00, 312.83it/s]

🎞️  영상 저장 완료 → D:\golfDataset\dataset\test\balanced_true\crop_keypoint\20201210_General_116_DOC_A_M30_MS_071_crop_skel.mp4
